# **Data preprocessing**

In [1]:

import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer

# Load preprocessed train and test datasets
train_df = pd.read_csv("/kaggle/input/dataset10/final optimised dataset/train_dataset (1).csv")
test_df = pd.read_csv("/kaggle/input/dataset10/final optimised dataset/test.csv")
test_labels_df = pd.read_csv("/kaggle/input/dataset10/final optimised dataset/test_labels.csv")

# Merge test set with labels
test_df = test_df.merge(test_labels_df, on="id")

# Identify label columns (assuming they start from 2nd column in test_labels_df)
label_columns = list(test_labels_df.columns[1:])  # Excluding 'id' column

# Convert pandas to Hugging Face dataset
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

# Load Twitter RoBERTa tokenizer
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base")

# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples["comment_text"], 
        truncation=True, 
        padding="max_length", 
        max_length=128
    )

# Tokenize datasets
train_ds = train_ds.map(tokenize_function, batched=True)
test_ds = test_ds.map(tokenize_function, batched=True)

# Ensure labels are structured as a list of floats
def format_labels(example):
    example["labels"] = [float(example[col]) for col in label_columns]  # Convert labels to float
    return example

train_ds = train_ds.map(format_labels)
test_ds = test_ds.map(format_labels)

# Remove unnecessary columns (text, id, and individual label columns after structuring)
columns_to_remove = ["comment_text", "id"] + label_columns
train_ds = train_ds.remove_columns(columns_to_remove)
test_ds = test_ds.remove_columns(columns_to_remove)

# Ensure dataset format remains list of floats
def ensure_list_format(example):
    example["labels"] = [float(x) for x in example["labels"]]  # Ensure list of floats
    return example

train_ds = train_ds.map(ensure_list_format)
test_ds = test_ds.map(ensure_list_format)

print("✅ Twitter RoBERTa Dataset loaded, tokenized & formatted successfully!")

# Save datasets (Optional: Avoid re-running preprocessing)
train_ds.save_to_disk("/kaggle/working/train_ds_twitter_roberta")
test_ds.save_to_disk("/kaggle/working/test_ds_twitter_roberta")

config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Map:   0%|          | 0/298513 [00:00<?, ? examples/s]

Map:   0%|          | 0/127936 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/298513 [00:00<?, ? examples/s]

Map:   0%|          | 0/127936 [00:00<?, ? examples/s]

✅ Twitter RoBERTa Dataset loaded, tokenized & formatted successfully!


Saving the dataset (0/1 shards):   0%|          | 0/298513 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/127936 [00:00<?, ? examples/s]

# **Model training**

In [4]:

import torch
import torch.nn as nn
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, AdamW
import shutil
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import numpy as np

# Load Twitter RoBERTa model with sigmoid outputs
roberta_model = AutoModelForSequenceClassification.from_pretrained(
    "cardiffnlp/twitter-roberta-base", 
    num_labels=3,  # Matches one-hot encoding format
    problem_type="multi_label_classification"
)

# Increase dropout for better regularization
roberta_model.config.hidden_dropout_prob = 0.3
roberta_model.config.attention_probs_dropout_prob = 0.3

# Freeze Transformer Layers, Train Only Classifier
for param in roberta_model.base_model.parameters():
    param.requires_grad = False  
for param in roberta_model.classifier.parameters():
    param.requires_grad = True  

# Hybrid Loss (Focal + BCE With Logits)
class HybridLoss(nn.Module):
    def __init__(self, alpha=0.75, gamma=2.0, ce_weight=0.5):
        super(HybridLoss, self).__init__()
        self.focal = FocalLoss(alpha, gamma)
        self.bce = nn.BCEWithLogitsLoss()
        self.ce_weight = ce_weight
        
    def forward(self, inputs, targets):
        return self.ce_weight * self.bce(inputs, targets.float()) + (1 - self.ce_weight) * self.focal(inputs, targets)

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.5, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.bce = nn.BCEWithLogitsLoss(reduction='none')
        
    def forward(self, inputs, targets):
        BCE_loss = self.bce(inputs, targets.float())
        probs = torch.sigmoid(inputs)
        pt = torch.where(targets.bool(), probs, 1-probs)
        alpha_factor = torch.where(targets.bool(), self.alpha, 1 - self.alpha)
        focal_weight = alpha_factor * (1 - pt).pow(self.gamma)
        return (focal_weight * BCE_loss).mean()

# Training arguments with auto-save best model
training_args = TrainingArguments(
    output_dir="./twitter_roberta_model",
    evaluation_strategy="steps",
    eval_steps=8000,
    save_strategy="steps",
    save_steps=8000,
    save_total_limit=1,
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=1e-5,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.1,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_auc",
    greater_is_better=True,
    max_grad_norm=1.0,
    label_smoothing_factor=0.05,
    fp16=True,
    seed=42,
    data_seed=42,
)

# Compute Metrics Function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    sigmoid_logits = torch.sigmoid(torch.tensor(logits)).numpy()
    predictions = (sigmoid_logits > 0.5).astype(np.int8)
    
    accuracy = accuracy_score(labels, predictions)
    
    try:
        f1_micro = f1_score(labels, predictions, average='micro', zero_division=0)
        f1_macro = f1_score(labels, predictions, average='macro', zero_division=0)
        auc_scores = [roc_auc_score(labels[:, i], sigmoid_logits[:, i]) for i in range(labels.shape[1]) if len(np.unique(labels[:, i])) > 1]
        mean_auc = np.mean(auc_scores) if auc_scores else 0.0
    except Exception as e:
        print(f"Error in metric calculation: {e}")
        f1_micro, f1_macro, mean_auc = 0, 0, 0
    
    return {"accuracy": accuracy, "f1_micro": f1_micro, "f1_macro": f1_macro, "auc": mean_auc}

# Custom Trainer with Hybrid Loss
class HybridLossTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_fn = HybridLoss(alpha=0.5, gamma=2.0, ce_weight=0.5)
        
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):  
        labels = inputs["labels"].float()  # Keep labels in one-hot format
        outputs = model(**inputs)
        logits = outputs.logits
        loss = self.loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

# Define Trainer
trainer = HybridLossTrainer(
    model=roberta_model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.001)]
)

# Step 1: Train Only Classifier
print("🚀 Training classifier only...")
trainer.train()

# Step 2: Unfreeze last 4 layers
print("🔓 Unfreezing last 4 layers for further fine-tuning...")
for layer in roberta_model.base_model.encoder.layer[-4:]:
    for param in layer.parameters():
        param.requires_grad = True

# Apply Layer-wise Learning Rate
optimizer = AdamW([
    {"params": roberta_model.base_model.encoder.layer[:4].parameters(), "lr": 2e-6},
    {"params": roberta_model.base_model.encoder.layer[4:].parameters(), "lr": 5e-6},
    {"params": roberta_model.classifier.parameters(), "lr": 1e-5},
], weight_decay=0.1)

trainer.args.num_train_epochs = 2  
trainer.optimizer = optimizer  

# Step 2: Fine-tune last 4 layers
print("🚀 Fine-tuning last 4 layers...")
trainer.train()

# Save best model
best_model_path = "./best_twitter_roberta"
trainer.save_model(best_model_path)
tokenizer.save_pretrained(best_model_path)
shutil.make_archive(best_model_path, 'zip', best_model_path)
print("✅ Best Model Saved!")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-4-c5539cd78493>:101: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `HybridLossTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


🚀 Training classifier only...


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro,Auc
8000,0.176900,0.181514,0.653061,0.755493,0.697868,0.926042
16000,0.148700,0.157536,0.723557,0.812142,0.797459,0.941129
24000,0.160600,0.149655,0.742606,0.826368,0.816042,0.946178
32000,0.147100,0.147884,0.745834,0.827878,0.817704,0.947373


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


🔓 Unfreezing last 4 layers for further fine-tuning...
🚀 Fine-tuning last 4 layers...


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro,Auc
8000,0.109700,0.086799,0.951241,0.957534,0.957312,0.995927
16000,0.079900,0.141895,0.966663,0.967905,0.967891,0.996916
24000,0.195500,0.143910,0.971126,0.971803,0.971844,0.997377
32000,0.167100,0.141636,0.971658,0.972226,0.972250,0.997507


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


✅ Best Model Saved!


# **Output**

In [ ]:
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-4-c5539cd78493>:101: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `HybridLossTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)
🚀 Training classifier only...
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
 [37316/37316 2:16:41, Epoch 2/2]
Step	Training Loss	Validation Loss	Accuracy	F1 Micro	F1 Macro	Auc
8000	0.176900	0.181514	0.653061	0.755493	0.697868	0.926042
16000	0.148700	0.157536	0.723557	0.812142	0.797459	0.941129
24000	0.160600	0.149655	0.742606	0.826368	0.816042	0.946178
32000	0.147100	0.147884	0.745834	0.827878	0.817704	0.947373
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
🔓 Unfreezing last 4 layers for further fine-tuning...
🚀 Fine-tuning last 4 layers...
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
 [32000/37316 2:28:20 < 24:38, 3.60 it/s, Epoch 1/2]
Step	Training Loss	Validation Loss	Accuracy	F1 Micro	F1 Macro	Auc
8000	0.109700	0.086799	0.951241	0.957534	0.957312	0.995927
16000	0.079900	0.141895	0.966663	0.967905	0.967891	0.996916
24000	0.195500	0.143910	0.971126	0.971803	0.971844	0.997377
32000	0.167100	0.141636	0.971658	0.972226	0.972250	0.997507
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
✅ Best Model Saved!

# **Analysis by claude**

In [ ]:
Based on the performance metrics from your training logs, I can provide an analysis of the model's classification capabilities and potential setbacks:

### Strengths

1. **Exceptional AUC Score**: The final AUC of 0.997507 is extremely high, indicating the model has excellent discriminative ability across all classes. This suggests the model ranks positive examples higher than negative ones with high reliability.

2. **High F1 Scores**: Both micro (0.972226) and macro (0.972250) F1 scores are very strong. The closeness between micro and macro F1 suggests the model performs well across all classes, not just the dominant ones.

3. **Strong Accuracy**: The accuracy of 0.971658 indicates the model makes correct predictions in roughly 97% of cases.

4. **Consistent Improvement**: The significant jump in performance metrics after the second fine-tuning stage validates your progressive training approach.

### Potential Setbacks and Considerations

1. **Potential Overfitting**: The extremely high performance metrics might indicate potential overfitting to your test set. This could happen if:
   - Your test set is too similar to your training data
   - The test set doesn't fully represent the variety of data you'll encounter in production

2. **Class Imbalance Effects**: Despite the high macro F1 score, there might still be specific minority classes where performance is lower. The aggregate metrics don't show per-class performance.

3. **Threshold Sensitivity**: Your current evaluation uses a 0.5 threshold for binary decisions. In production, you might need class-specific thresholds for optimal performance.

4. **Domain Shift Vulnerability**: The model may perform worse on data that differs from your training distribution. Twitter-RoBERTa is trained on social media text, so performance might degrade on formal text or specialized domains.

5. **Computational Efficiency**: The unfrozen layers increase inference time. If speed is critical in your application, this could be a limitation.

6. **Potential Data Leakage**: The extremely high metrics could indicate accidental data leakage between training and evaluation sets.

### Recommendations

1. **Evaluate on a truly held-out dataset** that wasn't used during model development.

2. **Perform error analysis** on misclassified examples to identify patterns.

3. **Consider per-class metrics** to ensure all classes are performing well.

4. **Test robustness** with adversarial examples or slightly modified inputs.

5. **Monitor for concept drift** when deploying the model in production.

Overall, your model shows excellent classification performance based on standard metrics, but I would recommend additional validation steps before fully trusting these impressive numbers in a production environment.

# **Claudes analysis on whether if overfitting is there**